In [ ]:
# 1. Imports & Paths
import importlib.util
import os, sys, re, pandas as pd
sys.path.append(os.path.join(os.pardir, "utils"))

SAVE_DIR = os.path.join(os.pardir, "data/embeddings")
DATA_DIR = os.path.join(os.pardir, "data")
resume_path = os.path.join(DATA_DIR, "resumes_cleaned.csv")
jobs_path   = os.path.join(DATA_DIR, "jobs_cleaned.csv")

In [ ]:
%load_ext autoreload
%autoreload 2
from utils import *


In [ ]:
# 2. Load Data
resume_df = pd.read_csv(resume_path)
job_posts_df = pd.read_csv(jobs_path)

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
resume_embeddings = model.encode(resume_df["Resume_clean"].tolist(), show_progress_bar=True)
job_embeddings = model.encode(job_posts_df["job_text_clean"].tolist(), show_progress_bar=True)

In [ ]:
# --- Save embeddings ---
np.save(os.path.join(SAVE_DIR, "resume_embeddings.npy"), resume_embeddings)
np.save(os.path.join(SAVE_DIR, "job_embeddings.npy"), job_embeddings)
